# 03 - Feature Engineering Client


# 03 — Feature Engineering Client (Olist)

Objectif :
Construire une table client agrégée (1 ligne = 1 client unique)
à partir des données transactionnelles nettoyées,
afin de permettre la segmentation et la modélisation prédictive.


# Chargement des tables nettoyés

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

PROJECT_DIR = Path.cwd().parent
INTERIM_DIR = PROJECT_DIR / "data" / "interim"


In [2]:
customers = pd.read_parquet(INTERIM_DIR / "customers_clean.parquet")
orders = pd.read_parquet(INTERIM_DIR / "orders_delivered.parquet")
order_items = pd.read_parquet(INTERIM_DIR / "order_items_clean.parquet")
payments = pd.read_parquet(INTERIM_DIR / "payments_clean.parquet")
reviews = pd.read_parquet(INTERIM_DIR / "reviews_clean.parquet")
products = pd.read_parquet(INTERIM_DIR / "products_clean.parquet")


# Vérification rapide

In [3]:
print("Customers :", customers.shape)
print("Orders    :", orders.shape)
print("Items     :", order_items.shape)
print("Payments  :", payments.shape)
print("Reviews   :", reviews.shape)
print("Products  :", products.shape)


Customers : (99441, 5)
Orders    : (96478, 8)
Items     : (112650, 7)
Payments  : (103886, 5)
Reviews   : (99224, 2)
Products  : (32951, 9)


# Creation de la table pivot

In [4]:
clients = customers[["customer_unique_id"]].drop_duplicates().copy()

clients.shape


(96096, 1)

# Jointures clients <> commandes

In [5]:
orders_clients = orders.merge(
    customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left"
)


In [7]:
print(orders_clients[["order_id", "customer_unique_id"]].head())


                           order_id                customer_unique_id
0  e481f51cbdc54678b7cc49136f2d6af7  7c396fd4830fd04220f754e42b4e5bff
1  53cdb2fc8bc7dce0b6741e2150273451  af07308b275d755c9edb36a90c618231
2  47770eb9100c2d0c44946d9cf07ec65d  3a653a41f6f9fc3d2a113cf8398680e8
3  949d5b44dbf5de918fe9c16f97b45f8a  7c142cf63193a1473d2e66489a9ae977
4  ad21c59c0840e6cb83a9ceb5573f8159  72632f0f9dd73dfee390c9b22eb56dd6


Les jointures ont été réalisées en LEFT JOIN afin de préserver
l’exhaustivité des transactions e-commerce,
garantissant qu’aucune commande livrée ne soit exclue
lors de la construction des variables client.


# Calcule du RFM

In [8]:
# On repart de la table orders_clients créée précédemment
orders_clients = orders.merge(
    customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left"
)


# Définir la date de référence (CRUCIAL)

In [9]:
reference_date = orders_clients["order_purchase_timestamp"].max()

reference_date


Timestamp('2018-08-29 15:00:37')

# Calcul de la RECENCY (R)

In [10]:
recency = (
    orders_clients
    .groupby("customer_unique_id")["order_purchase_timestamp"]
    .max()
    .reset_index()
)

recency["recency"] = (reference_date - recency["order_purchase_timestamp"]).dt.days

recency = recency[["customer_unique_id", "recency"]]


# Calcul de la FREQUENCY (F)

In [11]:
frequency = (
    orders_clients
    .groupby("customer_unique_id")["order_id"]
    .nunique()
    .reset_index(name="frequency")
)


# Calcul du MONETARY (M)

In [12]:
payments_per_order = (
    payments
    .groupby("order_id")["payment_value"]
    .sum()
    .reset_index()
)


# Jointure paiements <> Clients

In [13]:
payments_clients = payments_per_order.merge(
    orders_clients[["order_id", "customer_unique_id"]],
    on="order_id",
    how="left"
)


# Calcul du Monetary

In [14]:
monetary = (
    payments_clients
    .groupby("customer_unique_id")["payment_value"]
    .sum()
    .reset_index(name="monetary")
)


# Fusion des 3 composantes RFM

In [ ]:
rfm = recency.merge(frequency, on="customer_unique_id", how="left")
rfm = rfm.merge(monetary, on="customer_unique_id", how="left")

rfm.head()


,customer_unique_id,recency,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19
2,0000f46a3911fa3c0805444483337064,536,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,320,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,287,1,196.89


# Vérifications essentielles (OBLIGATOIRES)

In [16]:
# Dimensions
rfm.shape


(93358, 4)

In [18]:
# Valeurs manquantes
print(rfm.isna().mean())


customer_unique_id    0.000000
recency               0.000000
frequency             0.000000
monetary              0.000011
dtype: float64


In [19]:
# Statistiques descriptives
print(rfm.describe())


            recency     frequency      monetary
count  93358.000000  93358.000000  93357.000000
mean     236.941773      1.033420    165.198772
std      152.591453      0.209097    226.314579
min        0.000000      1.000000      9.590000
25%      113.000000      1.000000     63.060000
50%      218.000000      1.000000    107.780000
75%      345.000000      1.000000    182.560000
max      713.000000     15.000000  13664.080000


# Features supllémentaires

### Panier moyen par client

- Le client achète-t-il peu mais cher ou ou souvent mais peu cher ?

In [20]:
rfm["avg_basket"] = rfm["monetary"] / rfm["frequency"]


In [ ]:
rfm[["monetary", "frequency", "avg_basket"]].head()


,monetary,frequency,avg_basket
0,141.90,1,141.90
1,27.19,1,27.19
2,86.22,1,86.22
3,43.62,1,43.62
4,196.89,1,196.89


### Nombre total d'articles achetés

- le client achète-t-il plusieurs produits par commandes ?

In [22]:
# Calcule intermédiare items <> clients
items_clients = order_items.merge(
    orders_clients[["order_id", "customer_unique_id"]],
    on="order_id",
    how="left"
)


In [23]:
# Agrégation au niveau client
total_items = (
    items_clients
    .groupby("customer_unique_id")["order_item_id"]
    .count()
    .reset_index(name="total_items")
)


### Délai moyen de livraison 

- Le client subit-il des délais longs ?

In [24]:
# Calcule du délai par commande
orders_clients["delivery_delay_days"] = (
    orders_clients["order_delivered_customer_date"]
    - orders_clients["order_purchase_timestamp"]
).dt.days


In [25]:
# Agrégation au niveau client (varaible très liée à la satisfaction du client)
avg_delivery_delay = (
    orders_clients
    .groupby("customer_unique_id")["delivery_delay_days"]
    .mean()
    .reset_index(name="avg_delivery_delay")
)


### Score moyen de satisfaction client 

- Le client est-il globalement satisfait ?

In [26]:
# Jointure reviews <> clients
reviews_clients = reviews.merge(
    orders_clients[["order_id", "customer_unique_id"]],
    on="order_id",
    how="left"
)


In [27]:
# Agrégation au niveau client
# certains clients n'ont pas d'avis donc les valeurs manquantes sont normales
avg_review_score = (
    reviews_clients
    .groupby("customer_unique_id")["review_score"]
    .mean()
    .reset_index(name="avg_review_score")
)


### Diversité des catégories achetées

- Le client achète-t-il dans plusieurs univers produits ?

In [28]:
# Items <> produits
items_products = order_items.merge(
    products[["product_id", "product_category_name"]],
    on="product_id",
    how="left"
)


In [29]:
# items <> clients
items_products_clients = items_products.merge(
    orders_clients[["order_id", "customer_unique_id"]],
    on="order_id",
    how="left"
)


In [30]:
# Agrégations
category_diversity = (
    items_products_clients
    .groupby("customer_unique_id")["product_category_name"]
    .nunique()
    .reset_index(name="nb_categories")
)


### FUSION DE TOUS LES FEATURES CLEINTS

In [31]:
client_features = (
    rfm
    .merge(total_items, on="customer_unique_id", how="left")
    .merge(avg_delivery_delay, on="customer_unique_id", how="left")
    .merge(avg_review_score, on="customer_unique_id", how="left")
    .merge(category_diversity, on="customer_unique_id", how="left")
)


### Vérifications finales

In [32]:
# Dimensions
client_features.shape


(93358, 9)

In [34]:
# Valeurs manquantes 
print(client_features.isna().mean().sort_values(ascending=False))


avg_review_score      0.006459
avg_delivery_delay    0.000086
avg_basket            0.000011
monetary              0.000011
customer_unique_id    0.000000
recency               0.000000
frequency             0.000000
total_items           0.000000
nb_categories         0.000000
dtype: float64


### Aperçu

In [36]:
print(client_features.head())


                 customer_unique_id  recency  frequency  monetary  avg_basket  total_items  avg_delivery_delay  \
0  0000366f3b9a7992bf8c76cfdf3221e2      111          1    141.90      141.90            1                 6.0   
1  0000b849f77a49e4a4ce2b2a4ca5be3f      114          1     27.19       27.19            1                 3.0   
2  0000f46a3911fa3c0805444483337064      536          1     86.22       86.22            1                25.0   
3  0000f6ccb0745a6a4b88665a16c9f078      320          1     43.62       43.62            1                20.0   
4  0004aac84e0df4da2b147fca70cf8255      287          1    196.89      196.89            1                13.0   

   avg_review_score  nb_categories  
0               5.0              1  
1               4.0              1  
2               3.0              1  
3               4.0              1  
4               5.0              1  


## Feature Engineering — Conclusion

La table client finale a été construite au niveau du client unique.
Elle combine :
- des indicateurs RFM
- des variables comportementales
- des indicateurs logistiques
- une mesure de la satisfaction client

Cette table constitue la base de la segmentation et de la modélisation prédictive.


### Savegarde

In [38]:
# Sauvegarde de la table client finale
client_features.to_parquet(
    INTERIM_DIR / "client_features.parquet",
    index=False
)

print(" client_features sauvegardé dans data/interim/")


 client_features sauvegardé dans data/interim/
